# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api, ML_pipeline
from Clust.clust.ML.tool import meta

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_',
    'ingestion_param_y':{
        "feature_list": "None"
    }
}

# forecasting
param2 = {
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

### (Optional) Get np.array data for inference

In [3]:
import numpy as np
np_data = np.load('infer_regression_array.npy') # infer_regression_array , infer_forecasting_array , infer_classification_array
infer_np = np_data[0]
infer_np.shape

(144, 24)

In [4]:
infer_np

array([[0.58695652, 0.26795707, 0.50409437, ..., 0.4841629 , 0.984375  ,
        0.69230769],
       [0.58373591, 0.26630584, 0.50248586, ..., 0.48944193, 0.91927083,
        0.70512821],
       [0.5805153 , 0.26630584, 0.50087736, ..., 0.49472097, 0.85416667,
        0.71794872],
       ...,
       [0.31642512, 0.26080176, 0.45340222, ..., 0.37782805, 0.484375  ,
        0.69230769],
       [0.31723027, 0.26245299, 0.45632677, ..., 0.3785822 , 0.52604167,
        0.66666667],
       [0.31803543, 0.26428768, 0.46198089, ..., 0.37933635, 0.56770833,
        0.64102564]])

# 2. Backend

In [5]:
# 1. Backend Parameter Setting
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [6]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
#model_meta_db_style ='document'

# for testing mongodb model
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)
# model_meta = meta.read_model_meta_by_db_style(model_meta_db_style, meta_file_name, model_name)

# scaler info for inference data preparation
params['scaler_info'] = model_meta['scaler_param']

# model & target data info for ML inference
params['model_info'] = model_meta['model_info']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']

In [7]:
model_meta['model_info']['train_parameter']['batch_size'] = 1

In [8]:
# 3. Data Preparation
# make inference data X, scaler y
infer_X, scaler_y = ML_api.infer_data_preparation(params, infer_np)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [9]:
infer_X.shape

(144, 24)

In [10]:
# model inference
prediction_result = ML_api.ML_inference(params, infer_X, scaler_y)

inference data shape: torch.Size([144, 24])

Start inference

** Dimension of result for inference dataset = (144,)


In [11]:
prediction_result

,value
0,5.220139
1,5.220182
2,5.220222
3,5.220267
4,5.220300
...,...
139,5.219998
140,5.219971
141,5.219995
142,5.219980
